In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [2]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("dataset/article_LT_Labeling(20200505-20200731).csv", sep=',')
test = pd.read_csv("dataset/article_SS_Labeling(20200505-20200731).csv", sep=',')

In [69]:
newcsv = pd.read_csv("dataset/article_SSG_Labeling(20200505-20200731).csv", sep=',')

In [70]:
del newcsv['id']
del newcsv['team']
del newcsv['date']
del newcsv['image']
del newcsv['positive']
del newcsv['negative']

newcsv

,contents,positive_or_negative
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1
1,"‘완봉승+개막전 10연패 저지’ 서폴드, 에이스 품격 보여줬다 [오!쎈 인천][OS...",1
2,"SK 킹엄, KBO리그 데뷔전서 7이닝 3실점…패전 위기[서울=뉴시스] 프로야구 S...",1
3,"'7회 2아웃에 깨졌다' 서폴드, 퍼펙트게임-노히터 모두 무산[마이데일리 = 인천 ...",0
4,"‘KBO 데뷔전’ 킹엄, 7이닝 3실점 호투에도 패전 위기 [오!쎈 인천][OSEN...",1
...,...,...
1144,"[일문일답]SK 새 외인 타자 화이트 ""터커-문찬종에 조언 받아, SK서 최선 다할...",1
1145,"SK 새 외국인 타자 화이트 입국 ""로맥과 경쟁? 합심할 것""강화도서 2주간 자가격...",1
1146,"SK 외국인타자 화이트 31일 입국 ""로맥과의 경쟁? 많이 배울 것""SK 와이번스에...",1
1147,SK 화이트 입국 “로맥과 동료되는 것 기대…터커·문찬종과 인연”[스포츠경향] SK...,1


In [72]:
result = newcsv['contents'].str.split('.')
result

0       [[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막...
1       [‘완봉승+개막전 10연패 저지’ 서폴드, 에이스 품격 보여줬다 [오!쎈 인천][O...
2       [SK 킹엄, KBO리그 데뷔전서 7이닝 3실점…패전 위기[서울=뉴시스] 프로야구 ...
3       ['7회 2아웃에 깨졌다' 서폴드, 퍼펙트게임-노히터 모두 무산[마이데일리 = 인천...
4       [‘KBO 데뷔전’ 킹엄, 7이닝 3실점 호투에도 패전 위기 [오!쎈 인천][OSE...
                              ...                        
1144    [[일문일답]SK 새 외인 타자 화이트 "터커-문찬종에 조언 받아, SK서 최선 다...
1145    [SK 새 외국인 타자 화이트 입국 "로맥과 경쟁? 합심할 것"강화도서 2주간 자가...
1146    [SK 외국인타자 화이트 31일 입국 "로맥과의 경쟁? 많이 배울 것"SK 와이번스...
1147    [SK 화이트 입국 “로맥과 동료되는 것 기대…터커·문찬종과 인연”[스포츠경향] S...
1148    ['입국' SK 화이트 "로맥 KBO에서 큰 성공, 내가 많이 배워야"[MD인터뷰]...
Name: contents, Length: 1149, dtype: object

In [73]:
result = result.apply(lambda x: pd.Series(x))
result

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...,SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...,신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...,코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...,"종전 가장 늦게 개막한 시즌은 1995년으로, 4월 15일에 출발했다",공교롭게도 39년 역사상 가장 오래 기다렸던 개막전은 39년 역사상 가장 짧게 ...,이날 인천 경기의 공식 소요 시간은 단 2시간 6분,오후 2시 1분에 개시해 오후 4시 7분에 종료됐다,"이는 역대 개막전 최단 시간 신기록으로, 종전 최단 시간이었던 2000년 4월 5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"‘완봉승+개막전 10연패 저지’ 서폴드, 에이스 품격 보여줬다 [오!쎈 인천][OS...",/ dreamer@osen,co,"kr [OSEN=인천, 길준영 기자] 한화 이글스 서폴드(30)가 개인 통산 두...",서폴드는 5일 인천 SK행복드림구장에서 열린 ‘2020 신한은행 SOL KBO리...,2009년 이후 개막전 승리가 없었던 한화는 서폴드의 활약에 힘입어 3-0 승...,11년 만에 개막전 승리다,이날 승리로 한화는 개막전 9연패에서 마침내 탈출했다,지난 시즌 서폴드는 개막전에 나섰지만 5⅔이닝 5피안타(1피홈런) 1탈삼진 3...,하지만 이날 호투로 왜 자신이 에이스인지를 스스로 증명했다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"SK 킹엄, KBO리그 데뷔전서 7이닝 3실점…패전 위기[서울=뉴시스] 프로야구 S...",(사진 = SK 와이번스 제공) [인천=뉴시스] 김희준 기자 = SK 와이번스의 ...,킹엄은 5일 인천 SK행복드림구장에서 열린 한화 이글스와의 2020 신한은행 쏠...,염경엽 SK 감독은 이날 경기를 앞두고 킹엄의 투구수를 100개 미만으로 정했고...,킹엄은 무난한 투구를 했음에도 SK 타선이 7회말까지 1안타 1볼넷으로 무득점에...,SK는 올 시즌 에이스 역할을 해줘야 하는 킹엄에 개막전 선발의 중책도 맡겼다,"SK는 2019시즌 원투펀치를 이뤘던 김광현, 앙헬 산체스가 모두 팀을 떠난 상황이다","김광현은 메이저리그(MLB) 세인트루이스 카디널스와 계약하며 팀을 떠났고, 산체스...",헨리 소사와 재계약도 포기한 SK는 외국인 투수를 모두 새 얼굴로 구성했다,킹엄과 리카르도 핀토가 그들이다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"'7회 2아웃에 깨졌다' 서폴드, 퍼펙트게임-노히터 모두 무산[마이데일리 = 인천 ...",서폴드는 5일 인천SK행복드림구장에서 벌어진 2020 신한은행 SOL KBO 리...,1회말 선두타자 노수광의 땅볼을 직접 잡아 아웃시키는 등 공 10개로 가볍게 삼...,3회말에도 마지막 타자 정현을 스탠딩 삼진으로 돌려세운 서폴드는 4회말 노수광-고...,"5회말에도 로맥, 한동민, 윤석민을 잡는데 공 10개면 충분했던 서폴드는 6회말...",7회말에도 2아웃까지 퍼펙트 행진을 이어간 서폴드는 최정과 풀카운트 승부를 벌였고...,이어 로맥에 좌중간 안타까지 맞아 노히터의 희망마저 사라졌다,그래도 무실점 행진은 이어갔다,"2사 1,3루 위기에서 한동민을 1루 땅볼로 잡은 것이다",서폴드는 7회까지 1피안타 1볼넷 무실점으로 호투하고 있으며 한화는 3-0으로 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"‘KBO 데뷔전’ 킹엄, 7이닝 3실점 호투에도 패전 위기 [오!쎈 인천][OSEN...",/ dreamer@osen,co,"kr [OSEN=인천, 길준영 기자] 시즌 개막전에서 KBO리그 데뷔전을 가진 ...",킹엄은 5일 인천 SK행복드림구장에서 열린 ‘2020 신한은행 SOL KBO리그...,1회초 선두타자 이용규를 2루수 땅볼로 잡아낸 킹엄은 정은원에게도 땅볼을 유도했...,시작부터 1사 2루 위기에 빠진 킹엄은 호잉을 중견수 뜬공으로 잡아내고 이성열을 ...,킹엄은 2회에도 선두타자 송광민에게 2루타를 맞아 단숨에 득점 위기에 몰렸다,이번에는 김태균에게 1타점 적시타를 얻어맞으며 선취점을 허용했다,"이후 최재훈에게 볼넷을 내줘 1사 1, 2루 위기가 계속됐지만 하주석에게 병살타를...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,"[일문일답]SK 새 외인 타자 화이트 ""터커-문찬종에 조언 받아, SK서 최선 다할...",화이트는 31일 새벽 인천국제공항을 통해 입국했다,검역 절차를 마친 뒤 자가 격리 숙소에 도착한 화이트는 2주 간의 격리 기간을 거...,화이트는 지난 16일 SK와 계약했다,닉 킹엄의 대체자를 찾던 SK는 투수 대신 수 년전부터 지켜봤던 내야수인 화이트가...,"우투우타인 화이트는 빅리그 통산 256경기서 타율 2할3푼6리, 26홈런 103타...","트리플A 통산 성적은 282경기 타율 3할1푼1리, 49홈런, 230타점이다","SK는 화이트가 최 정, 제이미 로맥, 한동민과 함께 중심타선의 무게감을 더할 것...",-입국 소감은,▶코로나19 상황에서 KBO리그로 이적한다는 것이 쉽지 않았다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145,"SK 새 외국인 타자 화이트 입국 ""로맥과 경쟁? 합심할 것""강화도서 2주간 자가격...",[SK와이번스 제공,재배포 및 DB금지] (서울=연합뉴스) 김경윤 기자 = 프로야구 SK 와이번스...,"화이트는 31일 인천국제공항을 통해 입국한 뒤 ""새로운 환경에서 선수 생활을 이...",화이트는 부상으로 이탈한 전 외국인 투수 닉 킹엄의 대체 선수로 SK에 입단했다,SK는 외국인 투수 영입에 어려움을 겪자 외국인 타자를 추가로 영입했다,"화이트는 기존 외국인 타자 제이미 로맥과 타격 스타일이 비슷한데, 수비 위치(1...",SK 내부에선 로맥에게 외야 수비를 맡기거나 두 선수 중 한 명을 지명타자로 쓰...,"화이트는 이런 상황에 관해 ""난 마이너리그에서 6~7개, 메이저리그에서는 4개 ...","이어 ""로맥과 경쟁보다는 합심해서 팀 성적에 도움을 주고 싶다""고 밝혔다",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1146,"SK 외국인타자 화이트 31일 입국 ""로맥과의 경쟁? 많이 배울 것""SK 와이번스에...",(SK 와이번스 제공) © 뉴스1 (서울=뉴스1) 이재상 기자 = SK 와이번스...,"화이트는 ""매 경기 팀의 승리를 목표로 할 것""이라면서 ""제이미 로맥과 경쟁하기...","KBO리그에 빨리 적응하는 것이 중요하다""고 말했다","SK는 지난 16일 외국인 투수 닉 킹엄을 웨이버 공시하고, 타자인 화이트 영입...",그간 한국행 비자 발급이 늦어지면서 다소 시간이 지체됐다,화이트는 입국 심사와 특별 검역을 진행한 후 선별진료소로 이동해 신종 코로나바이...,이날 오전 한국에 도착한 화이트는 구단을 통해 소감을 전했다,"그는 ""코로나19 상황에서 KBO리그로 이적하는게 쉽지 않았다""며 ""KBO리그에...","긴 비행이었지만 한국에 무사히 도착해서 너무 기쁘다""고 말했다",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1147,SK 화이트 입국 “로맥과 동료되는 것 기대…터커·문찬종과 인연”[스포츠경향] SK...,sK와이번스 제공 SK의 대체 외인 타일러 화이트(30)가 31일 오전 인천국제...,화이트는 이날 입국 후 구단 홍보팀을 통해 진행한 인터뷰에서 “몇 년 전부터 외.

In [74]:
result.stack()

0     0     [포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...
      1       한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...
      2      SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...
      3       신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...
      4      코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...
                                  ...                        
1148  44                                    kr)  - ⓒ마이데일리(www
      45                                              mydaily
      46                                                   co
      47                                                  kr)
      48                                       무단전재&재배포 금지 - 
Length: 31394, dtype: object

In [75]:
result=result.stack().reset_index(level=1, drop=True).to_frame('sentence')

In [76]:
result = newcsv.merge(result, left_index=True, right_index=True, how='right')
result

,contents,positive_or_negative,sentence
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전..."
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1,한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1,SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1,신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...
0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1,코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...
...,...,...,...
1148,"'입국' SK 화이트 ""로맥 KBO에서 큰 성공, 내가 많이 배워야""[MD인터뷰][...",1,kr) - ⓒ마이데일리(www
1148,"'입국' SK 화이트 ""로맥 KBO에서 큰 성공, 내가 많이 배워야""[MD인터뷰][...",1,mydaily
1148,"'입국' SK 화이트 ""로맥 KBO에서 큰 성공, 내가 많이 배워야""[MD인터뷰][...",1,co
1148,"'입국' SK 화이트 ""로맥 KBO에서 큰 성공, 내가 많이 배워야""[MD인터뷰][...",1,kr)


In [77]:
del result['contents']

In [78]:
result

,positive_or_negative,sentence
0,1,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전..."
0,1,한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...
0,1,SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...
0,1,신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...
0,1,코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...
...,...,...
1148,1,kr) - ⓒ마이데일리(www
1148,1,mydaily
1148,1,co
1148,1,kr)


In [79]:
result.to_csv('C:/Users/DeepLearning_1/WindUp/dataset/sentence_ssg.csv', sep=',', encoding='utf-8')

In [13]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("dataset/sentence_train.csv", sep=',')
test = pd.read_csv("dataset/sentence_test.csv", sep=',')

print(train.shape)
print(test.shape)

(31178, 3)
(25562, 3)


In [18]:
# 훈련셋의 앞부분 출력
test.head(10)

,positive_or_negative,sentence
0,0,“대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...
1,0,인천 | 이석우 기자 5월5일
2,0,어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 ...
3,0,마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다
4,0,연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메...
5,0,지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸...
6,0,대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다
7,0,삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다
8,0,귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다
9,0,수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았...


In [17]:
del test['Unnamed: 0']

In [20]:
# 리뷰 문장 추출
sentences = train['sentence']
sentences[:10]

0    "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타...
1     /사진=한동훈 기자 롯데 자이언츠 새 외국인타자 딕슨 마차도(28)가 개막전부터 ...
2      마차도는 5일 수원 KT위즈파크에서 열린 2020 KBO리그 개막전 KT 위즈전...
3                                    롯데는 7-2로 역전승을 거뒀다
4      타격코치 출신 서용빈 해설위원이 시즌 전 "마차도의 장타력을 주목해야 한다"고 ...
5      서용빈 위원은 개막에 앞서 실시한 스타뉴스 설문조사에서 마차도를 눈여겨 봐야 한...
6     두산 페르난데스나 SK 로맥, KT 로하스 등 검증된 선수가 아니라 마차도를 콕 찍었다
7                   서 위원은 최근 마차도의 장타력이 눈에 띄게 발전했다고 짚었다
8      서 위원은 "마차도가 마이너리그에서 2015년부터 2018년까지 홈런을 11개를 쳤다
9                                 그런데 2019년에만 17개를 때렸다
Name: sentence, dtype: object

In [21]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타뉴스 수원=한동훈 기자] 롯데 마차도가 5일 수원 KT전 승리 후 수훈선수 인터뷰를 하고 있다 [SEP]',
 '[CLS]  /사진=한동훈 기자 롯데 자이언츠 새 외국인타자 딕슨 마차도(28)가 개막전부터 미친 존재감을 발휘했다 [SEP]',
 '[CLS]   마차도는 5일 수원 KT위즈파크에서 열린 2020 KBO리그 개막전 KT 위즈전에 7번 타자 겸 유격수로 선발 출전, 4타수 2안타 1홈런 4타점을 기록했다 [SEP]',
 '[CLS]  롯데는 7-2로 역전승을 거뒀다 [SEP]',
 '[CLS]   타격코치 출신 서용빈 해설위원이 시즌 전 "마차도의 장타력을 주목해야 한다"고 내다봤는데 예상이 개막전부터 적중한 셈이다 [SEP]',
 '[CLS]   서용빈 위원은 개막에 앞서 실시한 스타뉴스 설문조사에서 마차도를 눈여겨 봐야 한다고 분석했다 [SEP]',
 '[CLS]  두산 페르난데스나 SK 로맥, KT 로하스 등 검증된 선수가 아니라 마차도를 콕 찍었다 [SEP]',
 '[CLS]  서 위원은 최근 마차도의 장타력이 눈에 띄게 발전했다고 짚었다 [SEP]',
 '[CLS]   서 위원은 "마차도가 마이너리그에서 2015년부터 2018년까지 홈런을 11개를 쳤다 [SEP]',
 '[CLS]  그런데 2019년에만 17개를 때렸다 [SEP]']

In [23]:
# 라벨 추출
labels = train['positive_or_negative'].values
labels

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [24]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타뉴스 수원=한동훈 기자] 롯데 마차도가 5일 수원 KT전 승리 후 수훈선수 인터뷰를 하고 있다 [SEP]
['[CLS]', '"', '장', '##타', '##력', '주', '##목', '"', '서', '##용', '##빈', '해', '##설', '##이', '찍', '##은', '마', '##차', '##도', ',', '개', '##막', '##전', '##부터', '미', '##친', '존', '##재', '##감', '[', '★', '##현', '##장', ']', '[', '스', '##타', '##뉴스', '수', '##원', '=', '한', '##동', '##훈', '기자', ']', '롯', '##데', '마', '##차', '##도가', '5일', '수', '##원', 'K', '##T', '##전', '승', '##리', '후', '수', '##훈', '##선', '##수', '인', '##터', '##뷰', '##를', '하고', '있다', '[SEP]']


In [25]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    107,   9657,  22695,  28143,   9689,  68055,    107,
         9425,  24974, 102360,   9960,  31928,  10739,   9728,  10892,
         9246,  23466,  12092,    117,   8857, 118907,  16617,  17655,
         9309,  55358,   9680,  36210, 105197,    164,   1861,  30842,
        13890,    166,    164,   9477,  22695,  90947,   9460,  14279,
          134,   9954,  18778,  75965,  60886,    166,   9208,  28911,
         9246,  23466,  68516,  44015,   9460,  14279,    148,  11090,
        16617,   9484,  12692,  10003,   9460,  75965,  18471,  15891,
         9640,  21876, 118999,  11513,  32487,  11506,    102,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [26]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [27]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  9706, 33305, 43283,  9954, 62342, 10731, 11513, 25605,  9706,
        17138, 54867, 10739,  9309, 17138, 10954, 89389,  8907, 17730, 14102,
        11018,  8996, 96404,  9524, 26737, 32855,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [28]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [30]:
# 리뷰 문장 추출
sentences = test['sentence']
sentences[:10]

0    “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...
1                                   인천 | 이석우 기자   5월5일
2     어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 ...
3                 마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다
4     연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메...
5      지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸...
6                   대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다
7                   삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다
8                 귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다
9      수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았...
Name: sentence, dtype: object

In [31]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다 [SEP]',
 '[CLS]  인천 | 이석우 기자   5월5일 [SEP]',
 '[CLS]  어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 그동안 수고하셨습니다 [SEP]',
 '[CLS]  마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다 [SEP]',
 '[CLS]  연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메시지였다 [SEP]',
 '[CLS]   지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸기 어려웠다 [SEP]',
 '[CLS]  대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다 [SEP]',
 '[CLS]  삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다 [SEP]',
 '[CLS]  귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다 [SEP]',
 '[CLS]   수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았던 개막이 다가왔다 [SEP]']

In [32]:
# 라벨 추출
labels = test['positive_or_negative'].values
labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [33]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다 [SEP]
['[CLS]', '[UNK]', '대', '##구', '힘', '##내', '##세', '##요', '[UNK]', '[UNK]', '기', '##억', '##할', '##게', '##요', '[UNK]', '[UNK]', '희', '##망', '주', '##는', '[UNK]', '야구', '[UNK]', '가', '돌', '##아', '##왔다', '##ㆍ', '##무', '##관', '##중', '개', '##막', '삼', '##성', '-', 'NC', '##전', '관', '##중', '##석', '##엔', '[UNK]', '함께', '극', '##복', '##을', '[UNK]', '현', '##수', '##막', 'ㆍ', '##코', '##로', '##나', '##19', '의', '##료', '##진', '응', '##원', '영', '##상', '##에', '선수', '##들', '##도', '[UNK]', '덕', '##분에', '[UNK]', '손', '##동', '##작', 'ㆍ', '##각', '구', '##장', '깜', '##짝', '시', '##구', '·', '재', '##치', '##있는', '플', '##래', '##카', '##드', '[UNK]', '야구', '##의', '계', '##절', '[UNK]', '환', '##영', '관', '##중', '##은', '없', '##어', '##도', '오', '##늘', '##은', '좋은', '날', 'SK', '치', '##어', '##리', '##더',

In [34]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    100,   9069,  17196,  10028,  31605,  24982,  48549,
          100,    100,   8932,  91837,  14843,  14153,  48549,    100,
          100,  10023,  89292,   9689,  11018,    100, 106603,    100,
         8843,   9091,  16985,  72995, 111836,  32537,  20595,  41693,
         8857, 118907,   9410,  17138,    118,  55838,  16617,   8900,
        41693,  40958,  86933,    100,  19653,   8925,  70915,  10622,
          100,   9978,  15891, 118907,   2070,  25517,  11261,  16439,
        54055,   9637,  38688,  18623,   9636,  14279,   9574,  14871,
        10530,  78930,  27023,  12092,    100,   9075, 110355,    100,
         9450,  18778,  38709,   2070,  66540,   8908,  13890,   8943,
       119236,   9485,  17196,    217,   9659,  18622,  84177,   9944,
        37388,  24206,  15001,    100, 106603,  10459,   8887,  58931,
          100,   9995,  30858,   8900,  41693,  10892,   9555,  12965,
        12092,   9580, 118762,  10892,  79633,   8985,  21275,   9779,
      

In [35]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [36]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,    100,   9069,  17196,  10028,  31605,  24982,  48549,    100,
           100,   8932,  91837,  14843,  14153,  48549,    100,    100,  10023,
         89292,   9689,  11018,    100, 106603,    100,   8843,   9091,  16985,
         72995, 111836,  32537,  20595,  41693,   8857, 118907,   9410,  17138,
           118,  55838,  16617,   8900,  41693,  40958,  86933,    100,  19653,
          8925,  70915,  10622,    100,   9978,  15891, 118907,   2070,  25517,
         11261,  16439,  54055,   9637,  38688,  18623,   9636,  14279,   9574,
         14871,  10530,  78930,  27023,  12092,    100,   9075, 110355,    100,
          9450,  18778,  38709,   2070,  66540,   8908,  13890,   8943, 119236,
          9485,  17196,    217,   9659,  18622,  84177,   9944,  37388,  24206,
         15001,    100, 106603,  10459,   8887,  58931,    100,   9995,  30858,
          8900,  41693,  10892,   9555,  12965,  12092,   9580, 118762,  10892,
         79633,   8985,  21275,   9779, 

In [37]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [38]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [39]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [40]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [41]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [42]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [43]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [44]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.enabled = False

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
       
        # Forward 수행                
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   500  of  3,508.    Elapsed: 0:01:13.
  Batch 1,000  of  3,508.    Elapsed: 0:02:24.
  Batch 1,500  of  3,508.    Elapsed: 0:03:35.
  Batch 2,000  of  3,508.    Elapsed: 0:04:46.
  Batch 2,500  of  3,508.    Elapsed: 0:05:57.
  Batch 3,000  of  3,508.    Elapsed: 0:07:08.
  Batch 3,500  of  3,508.    Elapsed: 0:08:19.

  Average training loss: 0.62
  Training epcoh took: 0:08:21

Running Validation...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Accuracy: 0.69
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  3,508.    Elapsed: 0:01:11.
  Batch 1,000  of  3,508.    Elapsed: 0:02:22.
  Batch 1,500  of  3,508.    Elapsed: 0:03:33.
  Batch 2,000  of  3,508.    Elapsed: 0:04:45.
  Batch 2,500  of  3,508.    Elapsed: 0:05:56.
  Batch 3,000  of  3,508.    Elapsed: 0:07:07.
  Batch 3,500  of  3,508.    Elapsed: 0:08:18.

  Average training loss: 0.57
  Training epcoh took: 0:08:20

Running Validation...
  Accuracy: 0.71
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...
  Batch   500  of  3,508.    Elapsed: 0:01:11.
  Batch 1,000  of  3,508.    Elapsed: 0:02:22.
  Batch 1,500  of  3,508.    Elapsed: 0:03:34.
  Batch 2,000  of  3,508.    Elapsed: 0:04:46.
  Batch 2,500  of  3,508.    Elapsed: 0:05:57.
  Batch 3,000  of  3,508.    Elapsed: 0:07:09.
  Batch 3,500  of  3,508.    Elapsed: 0:08:20.

  Average training loss: 0.51
  Training epcoh took: 0:08:21

Running Validation..

In [45]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(),
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   100  of  3,196.    Elapsed: 0:00:03.
  Batch   200  of  3,196.    Elapsed: 0:00:07.
  Batch   300  of  3,196.    Elapsed: 0:00:10.
  Batch   400  of  3,196.    Elapsed: 0:00:13.
  Batch   500  of  3,196.    Elapsed: 0:00:16.
  Batch   600  of  3,196.    Elapsed: 0:00:20.
  Batch   700  of  3,196.    Elapsed: 0:00:23.
  Batch   800  of  3,196.    Elapsed: 0:00:26.
  Batch   900  of  3,196.    Elapsed: 0:00:29.
  Batch 1,000  of  3,196.    Elapsed: 0:00:33.
  Batch 1,100  of  3,196.    Elapsed: 0:00:36.
  Batch 1,200  of  3,196.    Elapsed: 0:00:39.
  Batch 1,300  of  3,196.    Elapsed: 0:00:43.
  Batch 1,400  of  3,196.    Elapsed: 0:00:46.
  Batch 1,500  of  3,196.    Elapsed: 0:00:49.
  Batch 1,600  of  3,196.    Elapsed: 0:00:52.
  Batch 1,700  of  3,196.    Elapsed: 0:00:56.
  Batch 1,800  of  3,196.    Elapsed: 0:00:59.
  Batch 1,900  of  3,196.    Elapsed: 0:01:02.
  Batch 2,000  of  3,196.    Elapsed: 0:01:05.
  Batch 2,100  of  3,196.    Elapsed: 0:01:09.
  Batch 2,200

In [46]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [47]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [87]:
logits = test_sentences(['한화 워윅 서폴드가 아깝게 실패했다'])
print(logits)
print(np.argmax(logits))

[[-1.6002097  1.8228879]]
1


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
